# twitterapiv2

This library offers access through Scala to the twitter API v2 endpoints. Additional aids for pagination and throttle management are also provided.

### Set up

You have a number of options to add the lib and dependencies to the classpath of this notebook:
1. Uber-jar
2. Classes (Windows path)
3. Classes (Windows/WSL path)
4. Classes (Docker path

To generate the uber-jar for the first option simply run the command `assembly` in the sbt prompt. To update the full class path files run the command `Compile/fullClasspath/exportToAmmoniteScript`in the sbt prompt. Choose your preferred option by uncommenting the corresponding line:

In [ ]:
// import $cp.target.`scala-2.13`.`twitterv2-0.1.jar`
import $file.`fullClasspath-Compile-WIN`

### Imports and dependencies

In [ ]:
import scala.concurrent.{Future, Await, ExecutionContext, duration}, duration._
import _root_.akka.actor.typed.ActorSystem
import _root_.akka.actor.typed.scaladsl.Behaviors
import scala.util.Success
import scala.util.Failure
import dev.habla.twitter.v2, v2.api._, v2.akka._

Common dependencies for actor-based systems and Akka stream:

In [ ]:
implicit val system = ActorSystem(Behaviors.empty, "TwitterV2")
implicit val ec = system.executionContext

Obtain the bearer token from the environment. Remember to pass this variable to docker, in case you started this notebook from there (`$ docker run -e <var_name> ...`)

In [ ]:
def bearerToken = scala.util.Properties.envOrElse("BEARER_TOKEN", "undefined")

### Search recent endpoint: single requests

We can access programmatically the [search/recent](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent) endpoint to obtain the response of a single request as follows:

In [ ]:
implicitly[HttpEndpoint[lookupt.Request]]

In [ ]:
val response: Future[recents.SingleResponse] = 
    recents.SingleRequest("scala3", bearerToken, max_results=Some(10)).single

In [ ]:
show(Await.result(response, 1.second))

### Search recent endpoint: pagination

But the library can also do pagination and throttle management for us automatically:

In [ ]:
val response: Future[recents.PaginatedResponse] = 
    v2.akka.recents.pagination.Run(recents.Pagination(recents.SingleRequest("scala3 -is:retweet", bearerToken), "output", Some(40)))